In [ ]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import transforms
from tqdm.notebook import tqdm

In [ ]:
#mean, std are passed as 0.5, 0.5 and image = (image - mean) / std
# Data transforms (normalization & data augmentation)
transform = transforms.Compose([transforms.RandomCrop(32, padding=2, padding_mode='reflect'), 
                                transforms.RandomHorizontalFlip(), 
                                transforms.ToTensor(),
                                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
train_set = torchvision.datasets.CIFAR10("./data", download=True, transform = transform)
test_set = torchvision.datasets.CIFAR10("./data", download=True, train=False, transform=transform)
train_set, val_set = torch.utils.data.random_split(train_set, [int(len(train_set)*0.8), int(len(train_set)*0.2)])

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
print(len(train_set), len(test_set), len(val_set))

40000 10000 10000


In [ ]:
train_load = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_load = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)
val_load = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=True)

In [ ]:
class Cnn(nn.Module): #class will inherit the nn.Module
  def __init__(self):
    super(Cnn, self).__init__()
    self.convolution = nn.Sequential(
      nn.Conv2d(3, 32, kernel_size=3, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(inplace=True),

      nn.Conv2d(32, 64, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2, stride=2),

      nn.Conv2d(64, 128, kernel_size=3, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(inplace=True),
      
      nn.Conv2d(128, 128, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Dropout2d(p=0.05),

      nn.Conv2d(128, 256, kernel_size=3, padding=1),
      nn.BatchNorm2d(256),
      nn.ReLU(inplace=True),

      nn.Conv2d(256, 256, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2, stride=2),
      
      #nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), 
      #nn.ReLU(),
      #nn.MaxPool2d(2, stride=2),
      #nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
      #nn.ReLU(),
      #nn.MaxPool2d(2, stride=2),
      #nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
      #nn.BatchNorm2d(256),
      #nn.ReLU(),
      #nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
      #nn.ReLU(),
      #nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1),
      #nn.ReLU(),
      #nn.MaxPool2d(2, stride=2),
    )
    self.linear = nn.Sequential(
      torch.nn.Linear(4096, 256),  
      nn.ReLU(),
      torch.nn.Linear(256, 1024),        
      nn.ReLU(),
      torch.nn.Linear(1024, 10),
      nn.BatchNorm1d(10),
      nn.LogSoftmax(dim = 1) #dimension
    )
  def forward(self, x):
    output = self.convolution(x)
    output = output.view(output.size(0), -1) # 256, 4, 4 to 4096 for 
    output = self.linear(output)
    return output

In [ ]:
Model = Cnn()
optimizer = optim.Adam(Model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

In [ ]:
if torch.cuda.is_available():
  Model.cuda()
torch.cuda.is_available()

True

In [ ]:
epochs = 20
last_val_loss = 100
for i in tqdm(range(epochs),desc='Epochs'):
  train_loss, val_loss = [], []
  train_correct, val_correct = 0, 0
  for images, labels in train_load:
    images, labels = images.cuda(), labels.cuda() # converting the data into GPU format
    optimizer.zero_grad() # clearing the Gradients of the model parameters
    outputs = Model(images)
    loss = criterion(outputs, labels)
    train_loss.append(loss.item())
    loss.backward()
    optimizer.step()
    predicted_labels = np.argmax(outputs.to("cpu").detach().numpy(), axis=1)
    train_correct += (predicted_labels == labels.to("cpu").detach().numpy()).sum()
  for images, labels in val_load:
    images, labels = images.cuda(), labels.cuda() # converting the data into GPU format
    outputs = Model(images)
    loss = criterion(outputs, labels)
    val_loss.append(loss.item())
    predicted_labels = np.argmax(outputs.to("cpu").detach().numpy(), axis=1)
    val_correct += (predicted_labels == labels.to("cpu").detach().numpy()).sum()
  train_accuracy = 100 * train_correct / len(train_set)
  val_accuracy = 100 * val_correct / len(val_set)
  train_loss = sum(train_loss)/len(train_loss)
  val_loss = sum(val_loss)/len(val_loss)
  if last_val_loss < val_loss and i >= 5:break # early stopping and patience
  last_val_loss = val_loss
  print('Epoch =', i, 'Validation Loss =', val_loss, 'validation Accuracy =', val_accuracy, 'Train Loss =', train_loss, 'Train Accuracy =', train_accuracy)

Epoch = 0 Validation Loss = 1.025336517649851 validation Accuracy = 67.25 Train Loss = 1.3140972292900086 Train Accuracy = 56.2325
Epoch = 1 Validation Loss = 0.8717875351571733 validation Accuracy = 72.02 Train Loss = 0.8946366575241089 Train Accuracy = 71.96
Epoch = 2 Validation Loss = 0.7237072120046919 validation Accuracy = 77.04 Train Loss = 0.7388201441287995 Train Accuracy = 76.705
Epoch = 3 Validation Loss = 0.6602797939139566 validation Accuracy = 78.57 Train Loss = 0.6399549149036408 Train Accuracy = 79.9975
Epoch = 4 Validation Loss = 0.6162099823070939 validation Accuracy = 80.45 Train Loss = 0.5692288183212281 Train Accuracy = 82.07
Epoch = 5 Validation Loss = 0.5846744022171968 validation Accuracy = 81.55 Train Loss = 0.5180821497917175 Train Accuracy = 83.74
Epoch = 6 Validation Loss = 0.5355778036603502 validation Accuracy = 82.99 Train Loss = 0.4744891436100006 Train Accuracy = 85.125
Epoch = 7 Validation Loss = 0.5125447902709815 validation Accuracy = 83.14 Train Loss

In [ ]:
correct=0
total_data=0
Model.eval()
for images, labels in test_load:
  images, labels = images.cuda(), labels.cuda()
  outputs = Model(images)
  predicted_labels = np.argmax(outputs.to("cpu").detach().numpy(), axis=1)
  correct += (predicted_labels == labels.to("cpu").detach().numpy()).sum()
accuracy = 100 * correct / len(test_set)
print("Test Accuracy =", accuracy)

Test Accuracy = 85.16
